# Scraping the French President's Official Agenda 

(L'agenda du Président de la République)

A day by day breakdown of what Macron has been up to since 2017 

Using **BeautifulSoup**

In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# the following list is courtesy of chatgpt 

months = ['mai-2017', 'juin-2017', 'juillet-2017', 'aout-2017', 'septembre-2017', 'octobre-2017', 'novembre-2017', 'decembre-2017',
'janvier-2018', 'fevrier-2018', 'mars-2018', 'avril-2018', 'mai-2018', 'juin-2018', 'juillet-2018', 'aout-2018',
'septembre-2018', 'octobre-2018', 'novembre-2018', 'decembre-2018', 'janvier-2019', 'fevrier-2019', 'mars-2019',
'avril-2019', 'mai-2019', 'juin-2019', 'juillet-2019', 'aout-2019', 'septembre-2019', 'octobre-2019', 'novembre-2019',
'decembre-2019', 'janvier-2020', 'fevrier-2020', 'mars-2020', 'avril-2020', 'mai-2020', 'juin-2020', 'juillet-2020',
'aout-2020', 'septembre-2020', 'octobre-2020', 'novembre-2020', 'decembre-2020', 'janvier-2021', 'fevrier-2021',
'mars-2021', 'avril-2021', 'mai-2021', 'juin-2021', 'juillet-2021', 'aout-2021', 'septembre-2021', 'octobre-2021',
'novembre-2021', 'decembre-2021', 'janvier-2022', 'fevrier-2022', 'mars-2022', 'avril-2022', 'mai-2022', 'juin-2022',
'juillet-2022', 'aout-2022', 'septembre-2022', 'octobre-2022', 'novembre-2022', 'decembre-2022', 'janvier-2023',
'fevrier-2023', 'mars-2023', 'avril-2023']


In [3]:
# this part I wrote - it goes through the agenda for each available month and
all_data = []

for month in months:
    url = f'https://www.elysee.fr/agenda-{month}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    sections = soup.find_all('section', {'class': 'container space-bottom-s'})

    for section in sections:
            date = section.find('span', {'class': 'text-up'}).text
            events = section.find_all('div', {'class': "list-table__content"})
            for event in events:
                item = {}
                time = event.find('span', {'class': 'list-table__hour'}).text
                desc = event.find('div', {'class': 'm-b-n'}).text
                try:
                    type = event.find('span', {'class': 'list-table__type'}).text
                except:
                    type = None
                item["date"] = date
                item["month"] = month
                item["time"] = time
                item["desc"] = desc
                item["type"] = type
                all_data.append(item)


In [4]:
with open('elysee.json', 'w') as f:
    json.dump(all_data, f, indent=4)

In [5]:
df = pd.DataFrame.from_dict(all_data)

In [6]:
# split the month column into year and month 

df['year'] = df['month'].str.split('-').str[1]
df['month'] = df['month'].str.split('-').str[0]

In [7]:
# replace the months with numbers

df['month'] = df['month'].replace({'janvier': '01', 'fevrier': '02', 'mars': '03', 'avril': '04', 'mai': '05', 'juin': '06', 'juillet': '07', 'aout': '08', 'septembre': '09', 'octobre': '10', 'novembre': '11', 'decembre': '12'})

In [8]:
# make a new column with the date in the format YYYY-MM-DD

df['date'] = df['year'] + '-' + df['month'] + '-' + df['date']


In [9]:
# save df to csv

df.to_csv('./testing_data/elysee.csv', index=False)

In [10]:
# this is what the final dataset looks like

df

,date,month,time,desc,type,year
0,2017-05-14,05,10h00,Passation de pouvoir et cérémonie d'investiture,None,2017
1,2017-05-14,05,12h00,Cérémonie à l'hôtel de Ville de Paris,None,2017
2,2017-05-17,05,20h00,"Dîner avec M. Donald Tusk, Président du Consei...",None,2017
3,2017-05-18,05,11h00,Conseil des ministres,None,2017
4,2017-05-18,05,14h30,Conseil de défense,None,2017
...,...,...,...,...,...,...
3747,2023-04-04,04,18h00,Déplacement à Clamart sur le thème de la sensi...,Déplacement,2023
3748,2023-04-05,04,Toute la journée,Visite d'État en République populaire de Chine.,Voyage officiel,2023
3749,2023-04-06,04,Toute la journée,Visite d'État en République populaire de Chine.,Voyage officiel,2023
3750,2023-04-07,04,Toute la journée,Visite d'État en République populaire de Chine.,Voyage officiel,2023
